<a href="https://colab.research.google.com/github/drosas1295/Ingen-IA/blob/main/02_Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [ ]:
from google.colab import files
files.upload()

Saving train.csv to train.csv


In [ ]:
!pip install kaggle
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle competitions download -c playground-series-s3e6
!unzip playground-series-s3e6.zip

playground-series-s3e6.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  playground-series-s3e6.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


## 1. Lectura del dataset

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,22725,55825,84,1,0,70,12031,3,10,2000,0,0,4477,786,345,0,0,5594137.1
22726,22726,65870,88,1,0,49,23197,9,9,2015,0,1,4811,2454,755,0,7,6594705.0
22727,22727,93192,42,1,0,39,8539,10,5,2014,1,0,5595,4072,789,0,0,9321511.4
22728,22728,65797,86,1,0,89,23197,2,10,2000,1,0,5358,2513,411,0,0,6584708.2


## 4. Preprocesado

#### 4.1 Simulación de datos faltantes

Realizando un analisis superficial al database, no se encuentran ajustes significativos a realizar en el preprocesado. Por motivos de aprendizaje y cumplir con los lineamientos para el database, se realizó una simulación de datos faltantes en 3 columnas, seleccionadas de manera aleatoria.

Para cumplir con el aspecto reproducible de la actividad, las columnas y sus porcentajes serán fijos.

Para realizar la selección aleatoria de las columnas y sus debidos porcentajes de datos faltantes, se utilizó el siguiente código:

```
train_mod = train.copy()

train_mod = train_mod.set_index('id')
train_mod = train_mod.drop(['price'], axis = 1)

columnas = np.random.choice(train_mod.columns, 3, replace = False)
print('Columnas seleccionadas aleatoriamente\n',columnas,"\n")

n_filas = len(train_mod)
for i in columnas:
  n_faltantes = int(n_filas * np.random.uniform(0.05,0.7))
  indices_faltantes = np.random.choice(n_filas, n_faltantes, replace=False)
  train_mod.loc[indices_faltantes,(i)] = np.nan
```
Dando como resultado los siguientes valores:
```
Columnas seleccionadas aleatoriamente
 ['hasStormProtector' 'cityCode' 'made'] 
```

Para visualizar los porcentajes faltantes se usó el siguiente código:
```
#Porcentaje de datos faltantes para cada variable del dataset
total           = train_mod.isnull().sum().sort_values(ascending=False)
percent         = (train_mod.isnull().sum()/train.isnull().count()*100).sort_values(ascending=False)
missing_train  = pd.concat([total,percent],axis=1,keys=["Total","Percent"])
missing_train
```
Dando como resultado los siguientes valores:
```
                    Total	Percent
hasStormProtector  13561.0	59.661241
made	            2278.0	10.021997
cityCode	        1953.0	8.592169
```

####A continuación se realiza el procedimiento correspondiente a replicar y reproducir estos resultados aleatorios.

In [ ]:
train_mod = train.copy()

columnas = ['hasStormProtector','cityCode','made']

n_filas = len(train_mod)
n_faltantes1 = (13561)
n_faltantes2 = (2278)
n_faltantes3 = (1953)
indices_faltantes1 = np.random.choice(n_filas, n_faltantes1, replace=False)
indices_faltantes2 = np.random.choice(n_filas, n_faltantes2, replace=False)
indices_faltantes3 = np.random.choice(n_filas, n_faltantes3, replace=False)
train_mod.loc[indices_faltantes1,(columnas[0])] = np.nan
train_mod.loc[indices_faltantes2,(columnas[1])] = np.nan
train_mod.loc[indices_faltantes3,(columnas[2])] = np.nan

In [ ]:
train_mod

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693.0,2,1,2000.0,0,1.0,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773.0,1,4,2000.0,0,1.0,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457.0,4,8,2020.0,1,NaN,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113.0,1,1,2000.0,0,NaN,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245.0,8,4,2018.0,1,0.0,7151,2786,863,0,7,6181908.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,22725,55825,84,1,0,70,12031.0,3,10,2000.0,0,0.0,4477,786,345,0,0,5594137.1
22726,22726,65870,88,1,0,49,23197.0,9,9,2015.0,0,1.0,4811,2454,755,0,7,6594705.0
22727,22727,93192,42,1,0,39,8539.0,10,5,2014.0,1,NaN,5595,4072,789,0,0,9321511.4
22728,22728,65797,86,1,0,89,23197.0,2,10,2000.0,1,NaN,5358,2513,411,0,0,6584708.2


In [ ]:
#Porcentaje de datos faltantes para cada variable del dataset
total           = train_mod.isnull().sum().sort_values(ascending=False)
percent         = (train_mod.isnull().sum()/train.isnull().count()*100).sort_values(ascending=False)
missing_train  = pd.concat([total,percent],axis=1,keys=["Total","Percent"])
missing_train

,Total,Percent
hasStormProtector,13561,59.661241
cityCode,2278,10.021997
made,1953,8.592169
isNewBuilt,0,0.000000
hasGuestRoom,0,0.000000
hasStorageRoom,0,0.000000
garage,0,0.000000
attic,0,0.000000
basement,0,0.000000
id,0,0.000000


#### 4.2 Eliminación de datos anómalos

Como habiamos mencionado en el primer notebook de exploración de datos, dentro del database existe una fecha de construcción anómala, correspondiente al año 10000. Por lo que eliminaremos todas las filas que contengan dicho año de construcción.

In [ ]:
anomalos = train_mod[train_mod['made'] == 10000].index.tolist()
train_mod = train_mod.drop(anomalos)
train_mod[train_mod['made'] == 10000]

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price


#### 4.3 Eliminación de datos con muchos datos faltantes

Debido a que la variable hasStromProtector posee mas del 50% de datos faltantes, se ha decidido eliminar dicha columna.

In [ ]:
print("Shape original: ", train_mod.shape)
train_mod = train_mod.drop(['hasStormProtector'], axis = 1)
print("Shape original: ", train_mod.shape)

Shape original:  (22725, 18)
Shape original:  (22725, 17)


#### 4.4 Relleno de datos faltantes

Para las variables que todavia poseen datos faltantes, rellenamos los mismos con la media de sus respectivas columnas.

In [ ]:
train_mod['cityCode'].fillna(train_mod.cityCode.median(), inplace=True)
train_mod['made'].fillna(train_mod.made.median(), inplace=True)

#Porcentaje de datos faltantes para cada variable del dataset
total           = train_mod.isnull().sum().sort_values(ascending=False)
percent         = (train_mod.isnull().sum()/train_mod.isnull().count()*100).sort_values(ascending=False)
missing_train  = pd.concat([total,percent],axis=1,keys=["Total","Percent"])
missing_train

,Total,Percent
id,0,0.0
made,0,0.0
hasGuestRoom,0,0.0
hasStorageRoom,0,0.0
garage,0,0.0
attic,0,0.0
basement,0,0.0
isNewBuilt,0,0.0
numPrevOwners,0,0.0
squareMeters,0,0.0
